In [27]:
import requests 
import time
import re
from datetime import datetime
from bs4 import BeautifulSoup,NavigableString, Comment
import lxml
def convert_string(input_str):
    # Regular expression pattern to match a date in the format dd-mm-yyyy
    pattern =r'\d+/\d+/\d+'
    
    # Search for the pattern in the string
    match = re.search(pattern, input_str)
    if match:
    # Extract and print the date if found
        date_str = match.group(0)
        date_obj = datetime.strptime(date_str, '%d/%m/%Y')
        date_str = date_obj.strftime('%Y-%m-%d')
        return date_str,date_obj
    else:
        return None

def get_datetime_obj(string):

    # Regular expression pattern to match the date
    pattern = r'\d{2}/\d{2}/\d{4}'
    
    # Search for the pattern in the string and extract the date
    match = re.search(pattern, string)
    if match:
        date_str = match.group(0)
        # Convert the extracted date string to datetime object
        date_obj = datetime.strptime(date_str, '%d/%m/%Y')
        date_str = date_obj.strftime('%Y-%m-%d')
    else:
        date_obj = None
        date_str = None
    
    return date_obj,date_str
def get_content_vtv(url):
    response = requests.get(url)
    time.sleep(3)
    soup = BeautifulSoup(response.content, 'html.parser')
    title = soup.find('h1').text.strip()
    date = soup.find('p', class_ = 'news-info').text.strip()
    published_date = get_datetime_obj(date)[1]
    first_paragraph = soup.find('h2', class_  = 'sapo')
    article = soup.find('div', id = 'entry-body')
    #strong_tag = soup.new_tag('strong') 
    #strong_tag.string = first_paragraph.text # Set the content of <i> tag
    # Insert the new element as the first child
    article.insert(0, first_paragraph)
    
    for script_or_style in article(['script', 'style','iframe','video',]):
                script_or_style.decompose()
    
    caption_text_list = article.find_all('div', class_ ='PhotoCMS_Caption')
    tags_to_remove = article.find_all(['a', 'span'])
    for tag in tags_to_remove:
        # Extract the text from the tag
        tag_text = tag.get_text()
        # Replace the tag with its text content
        tag.replace_with(tag_text)
        tag.text.strip()
    #remove all image attributes except somes from list
    list_attr = ['src','alt','data-original']
    for i in article.find_all('img'):
        for j in list(i.attrs.keys()):
            if j not in list_attr:
                i.attrs.pop(j)
    img_list = article.find_all('img')
    n_img = len(img_list)
    for i in range(0,n_img):
           
        caption_start = NavigableString("[caption id=\"\" align=\"aligncenter\" width=\"800\"]")
        try:
            caption_text = NavigableString(caption_text_list[i].find('p').get_text())
        except (IndexError,AttributeError):
            continue
        caption_end = NavigableString("[/caption]")
        # Insert the custom tags and caption text around the <img> tag
        img_list[i].insert_before(caption_start)
        img_list[i].insert_after(caption_end)
        img_list[i].insert_after(caption_text) 
    for i in article.find_all('img'):
        try:
            i['src'] = i['data-src']
        except KeyError:
            continue
    for i in caption_text_list:
        i.decompose()
    # add avatar photo on top
    first_photo = soup.find('img', class_ = 'news-avatar')
    try:
        article.insert(0, first_photo)
    except ValueError as e:
        print(e)
    # centering the image
    for i in article.find_all('img'):
        i['class'] = "aligncenter"
        #i['width'] = 800
       # i['height'] = 400
        del i['data-src']
    list_attr = ['src','alt','data-ogiginal','class']
    for i in article.find_all(recursive = True):
        for j in list(i.attrs.keys()):
            if j not in list_attr:
                i.attrs.pop(j)
    #remove empty div or empty space from element
    for item in article.find_all('div'):
        if item.string =="":
            item.decompose()
    for element in article.find_all(recursive = True,string=True):
        if isinstance(element, NavigableString) and element.strip() == '':
            element.extract()
    #remove html comment from element   
    article.find_all('p',recursive = True)[-1].decompose()
    for comment in article.find_all(string=lambda text: isinstance(text, Comment)):
        comment.extract()
    
    #article.find_all(recursive = True)[-1].decompose()
    #article.find('div', class_ ='width_common box-tinlienquanv2').decompose()
    source_tag = soup.new_tag('i') 
    source_tag.string = "Nguồn: vtv.vn"  # Set the content of <i> tag
    article.append(source_tag)
    return article, title, published_date
def get_post(url):
    try:
        content,title,published_date = get_content_vtv(url)
        return content,title,published_date
    except AttributeError as e:
        print(e)
def get_list_url(url):
    response = requests.get(url)
    soup = BeautifulSoup(response.content, 'html.parser')
    items = soup.find_all('h3')
    urls = []
    for i in items:
        path = i.find('a')['href']
        url = 'https://vtv.vn/'+ path
        urls.append(url)
    return urls
def filter_list(urls):
    filtered_urls = []
    crawl_time = datetime.fromtimestamp(time.time()-1*24*3600)
    for i in range(0,len(urls)):
        try:
            published_date = get_content_vtv(urls[i])[2]
            date_posted_norm = datetime.strptime(published_date, '%Y-%m-%d')
            if ( (date_posted_norm.day == crawl_time.day) and (date_posted_norm.month == crawl_time.month) and (date_posted_norm.year == crawl_time.year) ):
                filtered_urls.append(urls[i])
                #print(i)
        except AttributeError as e:
            print(e)
            continue
    return filtered_urls
#add list url to json
#add list url to json
def add_list(web_json_obj):
    for i in list(web_json_obj['urls'].keys()):
        for j in list(web_json_obj['urls'][i]['sub-category'].keys()):  
            urls = get_list_url(web_json_obj['urls'][i]['sub-category'][j]['url'])
            print(i,j,web_json_obj['urls'][i]['sub-category'][j]['url'])
            web_json_obj['urls'][i]['sub-category'][j]['url_list'] = filter_list(urls)
# add post content from get content function to json object
# add post content from get content function to json object
def add_post(web_json_obj):
    for i in list(web_json_obj['urls'].keys()):
        for j in list(web_json_obj['urls'][i]['sub-category'].keys()):
            web_json_obj['urls'][i]['sub-category'][j]['content'] = {}
            list_key = [v for v in range(0,len(web_json_obj['urls'][i]['sub-category'][j]['url_list']))]
            for u in list_key:
                web_json_obj['urls'][i]['sub-category'][j]['content'][u] = {}
                if u != "":
                    web_json_obj['urls'][i]['sub-category'][j]['content'][u]['text'] ,web_json_obj['urls'][i]['sub-category'][j]['content'][u]['title'],web_json_obj['urls'][i]['sub-category'][j]['content'][u]['published_date'] = get_post(web_json_obj['urls'][i]['sub-category'][j]['url_list'][u])
                    print(i,j,web_json_obj['urls'][i]['sub-category'][j]['cate_id'],web_json_obj['urls'][i]['sub-category'][j]['name'],web_json_obj['urls'][i]['sub-category'][j]['name'],web_json_obj['urls'][i]['sub-category'][j]['content'][u]['title'],web_json_obj['urls'][i]['sub-category'][j]['url_list'][u])
#add all necessary information to json object
def get_news_vtv():
    _vtv= {
            "home_page":"https://vtv.vn/",
            "urls":{
                "Quần vọt":
                {
                 "url":"https://vtv.vn/the-thao/tennis.htm#",
                 "sub-category":{  
                    0:{"name":"Tennis",
                     "url":"https://vtv.vn/the-thao/tennis.htm",
                     "cate_id":57,
                      "url_list" : []},
                 }
                }
            }
        }
#
    add_list(_vtv)
    add_post(_vtv)
    return _vtv
#send post content to wordpress via endpoint
def send_post_to_5goals(title,content,category_id,published_date):
    # URL of the API endpoint (this is a placeholder and needs to be replaced with the actual URL)
    url = "https://api2023.5goal.com/wp-json/custom/createPost"
    
    # Data to be sent in the POST request
    data = {
        "title": title,
        "content": content,
        "category_id": category_id,
        "token": 'draftpost',#'5goalvodichcmnl',  # Replace with your actual access token
        "published_date": published_date,
        "domain":"vtv"
          # Replace with the actual category ID as required
    }
    # Sending the POST request
    response = requests.post(url, data=data)
    
    # Checking the response
    if response.status_code == 200:
        print("The post was successfully created.")
        print("Response:", response.text)  # Prints the response text from the server
    else:
        print(f"Failed to create the post. Status code: {response.status_code}")
def main():
    _vtv = get_news_vtv()
    for i in [list(_vtv['urls'].keys())[0]]:
    #web_24h_com_vn2['url'][i]['cate_id']
        for j in list(_vtv['urls'][i]['sub-category']):
            url_list =  _vtv['urls'][i]['sub-category'][j]['url_list']
            print(url_list)
            for t in range(0,len(url_list)):
                content = _vtv['urls'][i]['sub-category'][j]['content'][t]['text']
                title = _vtv['urls'][i]['sub-category'][j]['content'][t]['title']
                published_date = _vtv['urls'][i]['sub-category'][j]['content'][t]['published_date']
                cate_id = _vtv['urls'][i]['sub-category'][j]['cate_id']
                print(title, url_list[t],content)
                #send_post_to_5goals(title,str(content), cate_id, published_date)
                time.sleep(2)


In [28]:
list_test = get_list_url('https://vtv.vn/the-thao/tennis.htm')

In [29]:
filter_list(list_test)

Cannot insert None into a tag.


['https://vtv.vn//tennis/rafael-nadal-va-nhung-ky-vong-cho-su-tro-lai-2023120410170502.htm',
 'https://vtv.vn//tennis/su-phat-trien-cua-cac-tay-vot-vo-dich-next-gen-atp-finals-20231204101144735.htm']

In [15]:
_vtv = get_news_vtv()

Quần vọt 0 https://vtv.vn/the-thao/tennis.htm


ValueError: Cannot insert None into a tag.

In [91]:
_vtv

{'home_page': 'https://vtv.vn/',
 'urls': {'Quần vọt': {'url': 'https://vtv.vn/the-thao/tennis.htm#',
   'sub-category': {0: {'name': 'Tennis',
     'url': 'https://vtv.vn/the-thao/tennis.htm',
     'cate_id': 57,
     'url_list': [],
     'content': {}}}}}}

In [20]:
test = get_list_url('https://vtv.vn/the-thao/tennis.htm')

In [8]:
test = get_content_vtv('https://vtv.vn/tennis/su-phat-trien-cua-cac-tay-vot-vo-dich-next-gen-atp-finals-20231204101144735.htm')

In [21]:
test

['https://vtv.vn//tennis/rafael-nadal-va-nhung-ky-vong-cho-su-tro-lai-2023120410170502.htm',
 'https://vtv.vn//tennis/su-phat-trien-cua-cac-tay-vot-vo-dich-next-gen-atp-finals-20231204101144735.htm',
 'https://vtv.vn//tennis/hamad-medjedovic-vo-dich-giai-quan-vot-next-gen-atp-finals-2023-20231203145837429.htm',
 'https://vtv.vn//tennis/arthur-fils-va-hamad-medjekovic-vao-chung-ket-atp-next-gen-finals-2023-20231202105645614.htm',
 'https://vtv.vn//tennis/luca-van-assche-co-tran-thang-thu-2-tai-next-gen-atp-finals-20231201095034892.htm',
 'https://vtv.vn//tennis/siet-chat-thoi-gian-giao-bong-tai-atp-next-gen-finals-20231128071727421.htm',
 'https://vtv.vn//tennis/hanh-trinh-dang-nho-cua-jannik-sinner-tai-davis-cup-2023-20231128043933467.htm',
 'https://vtv.vn//tennis/tong-giai-thuong-next-gen-atp-finals-len-toi-2-trieu-usd-20231128035205244.htm',
 'https://vtv.vn//tennis/dt-italia-lan-thu-2-vo-dich-davis-cup-20231127075101317.htm',
 'https://vtv.vn//tennis/dt-australia-vao-chung-ket-davi

In [11]:
datetime.strptime(test[2], '%Y-%m-%d')

datetime.datetime(2023, 12, 4, 0, 0)

In [7]:
get_datetime_obj('Cập nhật 10:11 ngày 04/12/2023')[0]

datetime.datetime(2023, 12, 4, 0, 0)

In [106]:
def filter_list(urls):
    filtered_urls = []
    crawl_time = datetime.fromtimestamp(time.time()-0*24*3600)
    for i in range(0,len(urls)):
        try:
            published_date = get_content_vtv(urls[i])[2]
            date_posted_norm = datetime.strptime(published_date,'%Y-%m-%d')
            if ( (date_posted_norm.day == crawl_time.day) and (date_posted_norm.month == crawl_time.month) and (date_posted_norm.year == crawl_time.year) ):
                filtered_urls.append(urls[i])
                #print(i)
        except AttributeError as e:
            print(e)
            continue
    return filtered_urls

In [104]:
get_datetime_obj('2023-12-04')

(None, None)

In [111]:
response = requests.get('https://vtv.vn/tennis/rafael-nadal-va-nhung-ky-vong-cho-su-tro-lai-2023120410170502.htm')
time.sleep(3)
soup = BeautifulSoup(response.content, 'html.parser')
title = soup.find('h1').text.strip()
date = soup.find('p', class_ = 'news-info').text.strip()
published_date = get_datetime_obj(date)[1]
first_paragraph = soup.find('h2', class_  = 'sapo')
article = soup.find('div', id = 'entry-body')
#strong_tag = soup.new_tag('strong') 
#strong_tag.string = first_paragraph.text # Set the content of <i> tag
# Insert the new element as the first child
article.insert(0, first_paragraph)

for script_or_style in article(['script', 'style','iframe','video',]):
            script_or_style.decompose()

caption_text_list = article.find_all('div', class_ ='PhotoCMS_Caption')
tags_to_remove = article.find_all(['a', 'span'])
for tag in tags_to_remove:
    # Extract the text from the tag
    tag_text = tag.get_text()
    # Replace the tag with its text content
    tag.replace_with(tag_text)
    tag.text.strip()
#remove all image attributes except somes from list
list_attr = ['src','alt','data-original']
for i in article.find_all('img'):
    for j in list(i.attrs.keys()):
        if j not in list_attr:
            i.attrs.pop(j)
img_list = article.find_all('img')
n_img = len(img_list)
for i in range(0,n_img):
       
    caption_start = NavigableString("[caption id=\"\" align=\"aligncenter\" width=\"800\"]")
    try:
        caption_text = NavigableString(caption_text_list[i].find('p').get_text())
    except (IndexError,AttributeError):
        continue
    caption_end = NavigableString("[/caption]")
    # Insert the custom tags and caption text around the <img> tag
    img_list[i].insert_before(caption_start)
    img_list[i].insert_after(caption_end)
    img_list[i].insert_after(caption_text) 
for i in article.find_all('img'):
    try:
        i['src'] = i['data-src']
    except KeyError:
        continue
for i in caption_text_list:
    i.decompose()
# add avatar photo on top
first_photo = soup.find('img', class_ = 'news-avatar')
article.insert(0, first_photo)
# centering the image
for i in article.find_all('img'):
    i['class'] = "aligncenter"
    #i['width'] = 800
   # i['height'] = 400
    del i['data-src']
list_attr = ['src','alt','data-ogiginal','class']
for i in article.find_all(recursive = True):
    for j in list(i.attrs.keys()):
        if j not in list_attr:
            i.attrs.pop(j)
#remove empty div or empty space from element
for item in article.find_all('div'):
    if item.string =="":
        item.decompose()
for element in article.find_all(recursive = True,string=True):
    if isinstance(element, NavigableString) and element.strip() == '':
        element.extract()
#remove html comment from element   
article.find_all('p',recursive = True)[-1].decompose()
for comment in article.find_all(string=lambda text: isinstance(text, Comment)):
    comment.extract()

#article.find_all(recursive = True)[-1].decompose()
#article.find('div', class_ ='width_common box-tinlienquanv2').decompose()
source_tag = soup.new_tag('i') 
source_tag.string = "Nguồn: vtv.vn"  # Set the content of <i> tag
article.append(source_tag)


In [112]:
article

<div class="content_detail ta-justify" id="entry-body"><img alt="" class="aligncenter" src="https://vtv1.mediacdn.vn/thumb_w/650/562122370168008704/2023/12/4/photo1701659799800-17016597999901359880287.jpg"/><h2 class="sapo">
                        VTV.vn - Sự kiện quần vợt được nhắc tới nhiều trong tuần qua chính là thông báo về sự trở lại của Rafael Nadal. Cựu số 1 thế giới đã nhận được nhiều kỳ vọng với kế hoạch tương lai.
                    </h2><p>Rafael Nadal xác nhận sẽ thi đấu ở giải quần vợt Brisbane mở rộng từ 31/12/2023 đến 7/1/2024. Đây là giải đấu đầu tiên của anh kể từ sau khi dừng bước tại vòng 2 Australia mở rộng 2023 do chấn thương cơ thắt lưng.<br/></p><p>Việc Rafael Nadal trở lại cũng nhận được sự quan tâm từ các đồng nghiệp. Tay vợt Taylor Fritz cho rằng: "Tôi nghĩ giờ chưa phải là lúc anh ấy nghĩ tới việc giải nghệ. Nadal vẫn còn thời gian thi đấu thêm nhiều năm và việc anh ấy trở lại rõ ràng là tin tốt cho mọi người. Tôi không nghĩ quá nhiều nếu phải so tài với a

NameError: name 'img_list' is not defined